In [3]:
pip install opencv-python numpy pyautogui pyaudio wave


  Using cached PyAutoGUI-0.9.54.tar.gz (61 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached PyMsgBox-1.0.9.tar.gz (18 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached pytweening-1.2.0.tar.gz (171 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'd

In [4]:
pip list

Package                   VersionNote: you may need to restart the kernel to use updated packages.

------------------------- --------------
anyio                     4.8.0
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 3.0.0
async-lru                 2.0.4
attrs                     25.1.0
babel                     2.16.0
beautifulsoup4            4.12.3
bleach                    6.2.0
certifi                   2024.12.14
cffi                      1.17.1
charset-normalizer        3.4.1
colorama                  0.4.6
comm                      0.2.2
debugpy                   1.8.12
decorator                 5.1.1
defusedxml                0.7.1
executing                 2.2.0
fastjsonschema            2.21.1
fqdn                      1.5.1
h11                       0.14.0
httpcore                  1.0.7
httpx                     0.28.1
idna                      3.10
ipykernel                 6.29.5
ipython     

In [2]:
import cv2
import numpy as np
import mss
import pyaudio
import wave
import threading
import time

In [10]:
def record_screen(stop_event, video_filename, fps=20.0, monitor_index=1):
    """
    화면을 캡처하여 영상 파일로 저장하는 함수입니다.
    
    Args:
        stop_event (threading.Event): 녹화 중지를 알리는 이벤트.
        video_filename (str): 저장할 영상 파일 이름 (예: "output_video.avi").
        fps (float): 초당 프레임 수.
        monitor_index (int): 캡처할 모니터 인덱스 (mss에서는 1부터 시작).
    """
    with mss.mss() as sct:
        # 기본 모니터를 선택 (여러 모니터가 있을 경우 index 선택)
        monitor = sct.monitors[monitor_index]
        width = monitor["width"]
        height = monitor["height"]

        # OpenCV VideoWriter 설정 (XVID 코덱 사용)
        fourcc = cv2.VideoWriter_fourcc(*"XVID")
        out = cv2.VideoWriter(video_filename, fourcc, fps, (width, height))
        
        print("화면 녹화를 시작합니다...")
        last_time = time.time()
        while not stop_event.is_set():
            # 모니터 전체 캡처
            img = np.array(sct.grab(monitor))
            # mss는 기본적으로 BGRA 형식이므로 BGR로 변환
            frame = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
            out.write(frame)
            
            # FPS를 맞추기 위한 딜레이 계산
            elapsed = time.time() - last_time
            sleep_time = max(0, 1/fps - elapsed)
            time.sleep(sleep_time)
            last_time = time.time()

        out.release()
        print("화면 녹화가 종료되었습니다.")

def record_audio(stop_event, audio_filename, channels=2, rate=44100, frames_per_buffer=1024):
    """
    마이크 입력을 녹음하여 WAV 파일로 저장하는 함수입니다.
    
    Args:
        stop_event (threading.Event): 녹음 중지를 알리는 이벤트.
        audio_filename (str): 저장할 오디오 파일 이름 (예: "output_audio.wav").
        channels (int): 채널 수.
        rate (int): 샘플링 레이트 (Hz).
        frames_per_buffer (int): 버퍼 크기.
    """
    p = pyaudio.PyAudio()
    stream = p.open(format=pyaudio.paInt16,
                    channels=channels,
                    rate=rate,
                    input=True,
                    frames_per_buffer=frames_per_buffer)
    frames = []
    print("오디오 녹음을 시작합니다...")
    while not stop_event.is_set():
        # exception_on_overflow=False로 설정하여 오버플로우 예외 방지
        data = stream.read(frames_per_buffer, exception_on_overflow=False)
        frames.append(data)

    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open(audio_filename, "wb")
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(pyaudio.paInt16))
    wf.setframerate(rate)
    wf.writeframes(b"".join(frames))
    wf.close()
    print("오디오 녹음이 종료되었습니다.")

def main():
    video_filename = "output_video.avi"
    audio_filename = "output_audio.wav"
    stop_event = threading.Event()

    # 별도의 스레드에서 화면 녹화와 오디오 녹음을 실행합니다.
    video_thread = threading.Thread(target=record_screen, args=(stop_event, video_filename))
    audio_thread = threading.Thread(target=record_audio, args=(stop_event, audio_filename))
    
    video_thread.start()
    audio_thread.start()
    
    print("녹화를 진행 중입니다. 중지하려면 Ctrl+C를 누르세요...")
    try:
        # 메인 스레드에서 대기
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        print("\n녹화를 중지합니다...")
        stop_event.set()
    
    video_thread.join()
    audio_thread.join()
    
    print("녹화가 완료되었습니다.")
    print(f"영상 파일: {video_filename}")
    print(f"오디오 파일: {audio_filename}")
    print("\n두 파일을 하나로 합치려면 ffmpeg 등을 이용할 수 있습니다. 예:")
    print(f"  ffmpeg -y -i {video_filename} -i {audio_filename} -c:v copy -c:a aac output_merged.mp4")

if __name__ == "__main__":
    main()


Exception in thread Thread-7 (record_screen):
Traceback (most recent call last):
  File "C:\anaconda3\envs\videoenv\Lib\threading.py", line 1041, in _bootstrap_inner
    self.run()
    ~~~~~~~~^^
  File "C:\anaconda3\envs\videoenv\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
    ~~~~~~~~~~~~~~~~~~~~~^^^^^^
  File "C:\anaconda3\envs\videoenv\Lib\threading.py", line 992, in run
    self._target(*self._args, **self._kwargs)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\손민구\AppData\Local\Temp\ipykernel_24512\3583464412.py", line 26, in record_screen
    img = pyautogui.screenshot()  # 화면 캡처
  File "C:\anaconda3\envs\videoenv\Lib\site-packages\pyautogui\__init__.py", line 228, in _couldNotImportPyScreeze
    raise PyAutoGUIException(
        "PyAutoGUI was unable to import pyscreeze. (This is likely because you're running a version of Python that Pillow (which pyscreeze depends on) doesn't support currently.) Please inst

🎥 녹화를 시작했습니다. 중지하려면 Enter를 누르세요...
 


✅ 화면과 오디오 녹화가 완료되었습니다.
✅ 최종 파일 저장 완료: final_output.mp4
